Random Forest Classifier 


Raw pixel values of the images without extracting features

two empty lists, images and labels, which will be used to store the flattened images and their corresponding labels (0 for cats and 1 for dogs), respectively.

In [1]:
import cv2
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from skimage import io
import shutil


In [2]:

# Paths to the directories containing cat and dog subsets
base_path = "/Users/blanchepenafiel/Desktop/Dataset"
train_path = os.path.join(base_path, "Train")
cat_subset_path = os.path.join(train_path, "train_cat_subset")
dog_subset_path = os.path.join(train_path, "train_dog_subset")
cat_val_path=os.path.join(base_path, "Val", "Cat")
dog_val_path=os.path.join(base_path, "Val", "Dog")

In [3]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Function to preprocess an image
def preprocess_image(image_path, target_size=(224, 224)):
    try:
        # Load image
        img = cv2.imread(image_path)
        if img is None:
            raise OSError("Failed to read image")
        
        # Resize image
        resized_img = cv2.resize(img, target_size)
        
        # Normalize image
        normalized_img = resized_img / 255.0
        
        # Flatten image
        flattened_img = normalized_img.flatten()
        
        return flattened_img
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Function to preprocess all images in a folder
def preprocess_images_in_folder(folder_path, target_size=(224, 224)):
    preprocessed_images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image(image_path, target_size)
            if preprocessed_image is not None:
                preprocessed_images.append(preprocessed_image)
    return np.array(preprocessed_images)

# Paths to the directories containing cat and dog subsets
base_path = "/Users/blanchepenafiel/Desktop/Dataset"
train_path = os.path.join(base_path, "Train")
cat_subset_path = os.path.join(train_path, "train_cat_subset")
dog_subset_path = os.path.join(train_path, "train_dog_subset")
cat_val_path=os.path.join(base_path, "Val", "Cat")
dog_val_path=os.path.join(base_path, "Val", "Dog")




In [4]:
# Preprocess cat images
cat_images = preprocess_images_in_folder(cat_subset_path)

# Preprocess dog images
dog_images = preprocess_images_in_folder(dog_subset_path)




Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 226 extraneous bytes before marker 0xd9


In [5]:
# Train a Random Forest model for each color channel (R, G, B)
colors = ['R', 'G', 'B']
models = {}

for color in colors:
    # Extract the corresponding channel
    if color == 'R':
        cat_channel = cat_images[:, :224*224]
        dog_channel = dog_images[:, :224*224]
    elif color == 'G':
        cat_channel = cat_images[:, 224*224:2*224*224]
        dog_channel = dog_images[:, 224*224:2*224*224]
    elif color == 'B':
        cat_channel = cat_images[:, 2*224*224:]
        dog_channel = dog_images[:, 2*224*224:]
    
    # Create training data and labels
    X_train = np.vstack((cat_channel, dog_channel))
    y_train = np.hstack((np.zeros(len(cat_channel)), np.ones(len(dog_channel))))
    
    # Train Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    models[color] = model



In [6]:
# Evaluate models on validation data
val_images = {'Cat': cat_val_path, 'Dog': dog_val_path}
for animal, path in val_images.items():
    for color, model in models.items():
        # Preprocess validation images
        val_images_preprocessed = preprocess_images_in_folder(path)
        
        # Extract the corresponding channel
        if color == 'R':
            val_channel = val_images_preprocessed[:, :224*224]
        elif color == 'G':
            val_channel = val_images_preprocessed[:, 224*224:2*224*224]
        elif color == 'B':
            val_channel = val_images_preprocessed[:, 2*224*224:]
        
        # Make predictions
        predictions = model.predict(val_channel)
        
        # Calculate accuracy
        accuracy = accuracy_score(np.ones(len(predictions)), predictions) if animal == 'Dog' else accuracy_score(np.zeros(len(predictions)), predictions)
        
        print(f"Accuracy for {animal} images using {color} channel: {accuracy}")

Accuracy for Cat images using R channel: 0.6586345381526104
Accuracy for Cat images using G channel: 0.6224899598393574
Accuracy for Cat images using B channel: 0.6767068273092369
Accuracy for Dog images using R channel: 0.5658291457286432
Accuracy for Dog images using G channel: 0.6351758793969849
Accuracy for Dog images using B channel: 0.5718592964824121
